This notebook will be mainly used for the capstone project!

In [50]:
import pandas as pd
import numpy as np

In [51]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [4]:
!conda install -c conda-forge beautifulsoup4 --yes

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
beautifulsoup4            4.6.3                    py35_0    conda-forge


In [5]:
!conda install -c conda-forge lxml --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    libxml2: 2.9.4-h6b072ca_5     --> 2.9.8-h422b904_2     conda-forge
    libxslt: 1.1.29-hcf9102b_5    --> 1.1.32-h88dbc4e_2    conda-forge
    lxml:    4.1.0-py35ha401a81_0 --> 4.2.5-py35hc9114bc_0 conda-forge

libxml2-2.9.8- 100% |################################| Time: 0:00:00  74.92 MB/s
libxslt-1.1.32 100% |################################| Time: 0:00:00  66.61 MB/s
lxml-4.2.5-py3 100% |################################| Time: 0:00:00  65.16 MB/s


In [55]:
from bs4 import BeautifulSoup
import requests

In [56]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

Create the CSV file

In [57]:
import csv
soup = BeautifulSoup(source, 'lxml')
csv_file = open('cms_scrape.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['PostalCode', 'Borough', 'Neighborhood'])

33

Parse/scrape the table from Wikipedia:

In [58]:
My_table = soup.find('table',{'class':'wikitable sortable'})
for tr in My_table.find_all('tr')[1:]:
    tds = tr.find_all('td')
    print(tds[0].text, tds[1].text, tds[2].text)
    csv_writer.writerow([tds[0].text, tds[1].text, tds[2].text])
csv_file.close()

M1A Not assigned Not assigned

M2A Not assigned Not assigned

M3A North York Parkwoods

M4A North York Victoria Village

M5A Downtown Toronto Harbourfront

M5A Downtown Toronto Regent Park

M6A North York Lawrence Heights

M6A North York Lawrence Manor

M7A Queen's Park Not assigned

M8A Not assigned Not assigned

M9A Etobicoke Islington Avenue

M1B Scarborough Rouge

M1B Scarborough Malvern

M2B Not assigned Not assigned

M3B North York Don Mills North

M4B East York Woodbine Gardens

M4B East York Parkview Hill

M5B Downtown Toronto Ryerson

M5B Downtown Toronto Garden District

M6B North York Glencairn

M7B Not assigned Not assigned

M8B Not assigned Not assigned

M9B Etobicoke Cloverdale

M9B Etobicoke Islington

M9B Etobicoke Martin Grove

M9B Etobicoke Princess Gardens

M9B Etobicoke West Deane Park

M1C Scarborough Highland Creek

M1C Scarborough Rouge Hill

M1C Scarborough Port Union

M2C Not assigned Not assigned

M3C North York Flemingdon Park

M3C North York Don Mills South


Drop the "Not assigned" rows from Borough coloumn

In [60]:
df=pd.read_csv('cms_scrape.csv')
df['Neighborhood']=df['Neighborhood'].str.replace('\n','')
df = df[df.Borough != 'Not assigned']

In [61]:
df.head(20)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


Group the similar PostalCode and Borough together

In [64]:
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df=df.sort_values(by='PostalCode',ascending=True)

Look for the 'Neighborhoods' with 'Not assigned' values

In [65]:
df.loc[df['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


Since there is only one Neighborhood wiht 'Not assigned' value, we shall simply replace that Neighborhood's value to match its Borough

In [66]:
df['Neighborhood']=df['Neighborhood'].replace('Not assigned', 'Queen\'s Park')

In [67]:
df.tail(20)

,PostalCode,Borough,Neighborhood
83,M6R,West Toronto,"Parkdale, Roncesvalles"
84,M6S,West Toronto,"Runnymede, Swansea"
85,M7A,Queen's Park,Queen's Park
86,M7R,Mississauga,Canada Post Gateway Processing Centre
87,M7Y,East Toronto,Business reply mail Processing Centre969 Eastern
88,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto"
89,M8W,Etobicoke,"Alderwood, Long Branch"
90,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
91,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
92,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."


In [69]:
df.shape

(103, 3)